## ひろしまQuest2020#stayhome【コース予測部門】

In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 300)

### 管理番号

In [95]:
model_No = 23

### sub-modelを使用するかどうか

In [96]:
use_sub_model = False

### n/4

In [97]:
sample_No = 4

In [98]:
ALL_MERGE = 'intermediate/{}/all_merge_{}_{}.f'.format(model_No, model_No, sample_No)
SUBMIT = 'submit/{}/course_{}_tune.csv'.format(model_No, model_No)
SUBMIT_F = 'submit/{}/course_{}_{}.f'.format(model_No, model_No, sample_No)

if use_sub_model:
    ALL_MERGE = 'intermediate/{}/all_merge_{}_{}_sub.f'.format(model_No, model_No, sample_No)
    SUBMIT = 'submit/{}/course_{}_sub.csv'.format(model_No, model_No)
else:
    OUT_SUBMODEL = 'intermediate/{}/course_predict_{}_{}.f'.format(model_No, model_No, sample_No)

In [99]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 190)

In [100]:
if use_sub_model:
    all_pitch['predict_curve'] = all_pitch['predict_curve'] / all_pitch['predict_straight']
    all_pitch['predict_slider'] = all_pitch['predict_slider'] / all_pitch['predict_straight']
    all_pitch['predict_shoot'] = all_pitch['predict_shoot'] / all_pitch['predict_straight']
    all_pitch['predict_fork'] = all_pitch['predict_fork'] / all_pitch['predict_straight']
    all_pitch['predict_changeup'] = all_pitch['predict_changeup'] / all_pitch['predict_straight']
    all_pitch['predict_sinker'] = all_pitch['predict_sinker'] / all_pitch['predict_straight']
    all_pitch['predict_cutball'] = all_pitch['predict_cutball'] / all_pitch['predict_straight']
    all_pitch.drop(columns=[
        'predict_0', 'predict_1', 'predict_2', 'predict_3', 'predict_4', 'predict_5', 'predict_6',
        'predict_7', 'predict_8', 'predict_9', 'predict_10', 'predict_11', 'predict_12',
       'predict_straight'
    ], inplace=True)

In [101]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 190)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0,120.0,0.159363,0.201195,0.219124,0.191235,0.229084,0.183267,0.239044,0.205179,0.181275,0.191235,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,66.0,16.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,0.200000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0.0,0.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0,120.0,0.159363,0.201195,0.219124,0.191235,0.229084,0.183267,0.239044,0.205179,0.181275,0.191235,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,66.0,16.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-

### train
- 行数: 257117

In [102]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 190)

#### 特徴量に加えた部分を抜く
- train_pitch_1 = train_pitch.query('index <= 60000')
- train_pitch_2 = train_pitch.query('index > 60000 & index <= 120000')
- train_pitch_3 = train_pitch.query('index > 120000 & index <= 180000')
- train_pitch_4 = train_pitch.query('index > 180000')

In [103]:
if sample_No == 1:
    train = train.query('index > 60000')
elif sample_No == 2:
    train = train.query('index <= 60000 | index > 120000')
elif sample_No == 3:
    train = train.query('index <= 120000 | index > 180000')
elif sample_No == 4:
    train = train.query('index <= 180000')
train.shape

(180001, 190)

### test
- 行数: 521650

In [104]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 190)

In [105]:
del all_pitch
gc.collect()

62

In [106]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [107]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

### LGBM学習
- multiclass : クラス数=13

In [108]:
lgb_param = {
    'objective' : 'multiclass',
    'boosting_type': 'gbdt',
    'metric' : 'multi_logloss',
    'num_class' : 13,
    'seed' : 0,
    'learning_rate' : 0.1,
    'lambda_l1': 8.769293390201968, 
    'lambda_l2': 3.913949617576324e-05, 
    'num_leaves': 6, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.8391111798378441, 
    'bagging_freq': 4, 
    'min_child_samples': 50
}

In [109]:
def lightgbm_cv(lgb_param, lgb_train):
    cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

    num_boost_round = len(cv_results['multi_logloss-mean'])
    print('Best num_boost_round:', num_boost_round)
    best_cv_score = cv_results['multi_logloss-mean'][-1]
    print('Best CV score:', best_cv_score)
    best_iter = int(num_boost_round * 1.1)
    return best_iter

### Feature Importance

In [110]:
def feature_importance(lgb_model):
    fi = lgb_model.feature_importance()
    fn = lgb_model.feature_name()
    df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
    df_feature_importance.sort_values('feat_imp', inplace=True)
    return df_feature_importance

In [111]:
%%time
lgb_train = lgb.Dataset(train_d, train['course'])
best_iter = lightgbm_cv(lgb_param, lgb_train)

[100]	cv_agg's multi_logloss: 2.35512 + 0.000928254
[200]	cv_agg's multi_logloss: 2.34895 + 0.00108167
[300]	cv_agg's multi_logloss: 2.34618 + 0.00112748
[400]	cv_agg's multi_logloss: 2.34473 + 0.00122232
[500]	cv_agg's multi_logloss: 2.34381 + 0.00129119
[600]	cv_agg's multi_logloss: 2.34328 + 0.00139189
[700]	cv_agg's multi_logloss: 2.34298 + 0.00138999
[800]	cv_agg's multi_logloss: 2.34273 + 0.00150337
[900]	cv_agg's multi_logloss: 2.34266 + 0.00147934
[1000]	cv_agg's multi_logloss: 2.3427 + 0.0015343
Best num_boost_round: 938
Best CV score: 2.3425899300822706
CPU times: user 1h 16min 8s, sys: 23.6 s, total: 1h 16min 32s
Wall time: 4min 47s


In [112]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 23min 23s, sys: 7.61 s, total: 23min 31s
Wall time: 1min 28s


In [113]:
feature_importance(lgb_model).tail(30)

,feat_name,feat_imp
124,height_bat,391
0,pitch_cnt_in_game,397
165,rank_x_year_dif_p-b,398
178,salary_year_dif_b-c,402
141,bat_game_cnt,409
125,weight_bat,415
103,slider,437
45,elapsed_from_inning,446
47,elapsed_batter,446
182,bmi_dif_b-c,457


### 予測

In [114]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 18s, sys: 984 ms, total: 6min 19s
Wall time: 25.3 s


## 結果出力

In [115]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.069210,0.143671,0.069583,0.040076,0.063694,0.039958,0.064302,0.055053,0.026323,0.179814,0.079634,0.095850,0.072833
1,1,0.059031,0.115584,0.064240,0.029140,0.049025,0.040886,0.046316,0.061257,0.027551,0.175430,0.099414,0.120577,0.111550
2,2,0.032732,0.082051,0.048944,0.021704,0.037047,0.028867,0.045764,0.053366,0.020658,0.158267,0.111225,0.219845,0.139530
3,3,0.051486,0.102603,0.059064,0.021728,0.033107,0.034125,0.045646,0.058814,0.030829,0.186184,0.104636,0.145762,0.126015
4,4,0.058449,0.112607,0.074254,0.032736,0.042020,0.050262,0.054816,0.068968,0.033378,0.140634,0.085410,0.125885,0.120580


### CSV出力

In [116]:
#submit.to_csv(SUBMIT, header=False, index=False)

### Feather出力

In [117]:
submit_f = submit.drop(columns=['index'])
submit_f.rename(columns={
        0: 'predict_0', 1: 'predict_1', 2: 'predict_2', 3: 'predict_3',
        4: 'predict_4', 5: 'predict_5', 6: 'predict_6', 7: 'predict_7',
        8: 'predict_8', 9: 'predict_9', 10: 'predict_10', 11: 'predict_11', 12: 'predict_12'
    }, inplace=True)
submit_f.to_feather(SUBMIT_F)
print(SUBMIT_F)

submit/23/course_23_4.f


### 結果まとめ

In [118]:
ave_result = (sample_No == 4)

In [119]:
if ave_result:
    result = 'submit/{}/course_{}_{}.f'.format(model_No, model_No, 1)
    print(result)
    df = pd.read_feather(result)
    columns = ['predict_0', 'predict_1', 'predict_2', 'predict_3', 
               'predict_4', 'predict_5', 'predict_6', 'predict_7',
              'predict_8', 'predict_9', 'predict_10', 'predict_11', 'predict_12']

    for i in range(2,5):
        result = 'submit/{}/course_{}_{}.f'.format(model_No, model_No, i)
        print(result)
        temp = pd.read_feather(result)
        for c in columns:
            df[c] = df[c] + temp[c]

    for c in columns:
        df[c] = df[c]/4

    # 出力
    df = df.reset_index()
    df.to_csv(SUBMIT, header=False, index=False)
    print(SUBMIT)

submit/23/course_23_1.f
submit/23/course_23_2.f
submit/23/course_23_3.f
submit/23/course_23_4.f
submit/23/course_23_tune.csv


### 球種予測で使用する

In [120]:
%%time
if not use_sub_model:
    train_predict = lgb_model.predict(train_d, num_iteration = lgb_model.best_iteration)

CPU times: user 2min 9s, sys: 1.02 s, total: 2min 10s
Wall time: 8.55 s


In [121]:
if not use_sub_model:
    df_train_predict = pd.DataFrame(train_predict).reset_index()
    submodel = pd.concat([df_train_predict, submit], ignore_index=True)
    submodel.drop(columns=['index'], inplace=True)
    submodel.rename(columns={
        0: 'predict_0', 1: 'predict_1', 2: 'predict_2', 3: 'predict_3',
        4: 'predict_4', 5: 'predict_5', 6: 'predict_6', 7: 'predict_7',
        8: 'predict_8', 9: 'predict_9', 10: 'predict_10', 11: 'predict_11', 12: 'predict_12'
    }, inplace=True)
    print(submodel.shape)
    submodel.head()

(701651, 13)


In [122]:
if not use_sub_model:
    submodel.to_feather(OUT_SUBMODEL)
    print(OUT_SUBMODEL)

intermediate/23/course_predict_23_4.f


### SIGNATE

In [123]:
'signate submit --competition-id=276 ./{} --note feat={}'.format(SUBMIT, column_cnt)

'signate submit --competition-id=276 ./submit/23/course_23_tune.csv --note feat=190'

### GCS

In [124]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/23/course_23_tune.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 23_tune(190) 
- 'learning_rate' : 0.1,
- 'num_leaves': 4,
- (1) Best CV score: 2.341850353687299
- (2) Best CV score: 2.3396210433383273
- (3) Best CV score: 2.341554253259382
- (4) Best CV score: 2.3425899300822706
- 評価結果  : 2.3493330

#### 23(190) ... 2017年の集計がない選手はNaNに
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- (1) Best CV score: 2.3410495769108106
- (2) Best CV score: 2.3389861210773724
- (3) Best CV score: 2.3409606333569144
- (4) Best CV score: 2.3417060924636233
- 評価結果  : 2.3486626 ★

#### 22_sub3(198) ... 球種の予測をそのまま使う
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- (1) Best CV score: 2.324145615655472
- (2) Best CV score: 2.320827785286294
- (3) Best CV score: 2.324029135378343
- (4) Best CV score: 2.3253020291900324
- 評価結果  : 2.3523530

#### 22_sub2(197) ... 特徴量で集計した部分を抜く
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- (1) Best CV score: 2.327086859587935
- (2) Best CV score: 2.3242695883810653
- (3) Best CV score: 2.326650154544137
- (4) Best CV score: 2.3279934162416356
- 評価結果  : 2.3510544

#### 22_sub(197)
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- (1) Best CV score: 2.320717948163731
- (2) Best CV score: 2.3204691031051947
- (3) Best CV score: 2.3217573750471856
- (4) Best CV score: 2.320525945710633
- 評価結果  : 2.3517300

#### 22(190)
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- (1) Best CV score: 2.3350431782409524
- (2) est CV score: 2.335302207655757
- (3) Best CV score: 2.3353277079714583
- (4) Best CV score: 2.3340380616284415
- 評価結果  : 2.3487659

#### 21_sub(198)
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 639
- Best CV score: 2.3159487785697754
- Wall time: 5min
- 評価結果  : 2.3572586

#### 20_sub(173)
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 793
- Best CV score: 2.315349281506766
- Wall time: 9min 57s
- 評価結果  : 2.3585163

#### 20(166) ... player同士の組み合わせを削除
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 572
- Best CV score: 2.3271237073894153
- Wall time: 6min 59s
- 評価結果  : 2.3552697

#### 19_sub(197) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 774
- Best CV score: 2.316818574234304
- Wall time: 10min 49s
- 評価結果  : 2.3581029

#### 19(190) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 550
- Best CV score: 2.327580481542456
- Wall time: 7min 36s
- 評価結果  : 2.3554862

#### 18_sub(225) ...サブモデルで球種の予測を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 548
- Best CV score: 2.308116414786115
- Wall time: 8min 25s
- 評価結果  : 2.3638683

#### 18(217) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 373
- Best CV score: 2.3204478805445055
- Wall time: 6min 9s
- 評価結果  : 2.3604751

#### 17(192) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 415
- Best CV score: 2.3200530570975535
- Wall time: 15min 10s
- 評価結果  : 2.3607863

#### 16(181)  ...選手IDごとの2017年のコース実績
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 355
- Best CV score: 2.3203553166835866
- Wall time: 5min 8s
- 評価結果  : 2.3608998

#### 15(153)  ...左右で球種/コースを別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 932
- Best CV score: 2.3387434142884445
- Wall time: 9min 41s
- 評価結果  : 2.3552543

#### 14(153)  ...左右で球種を別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 909
- Best CV score: 2.340082741044867
- Wall time: 9min 40s
- 評価結果  : 2.3555548

#### 3(157)  ...前の投球からの時間差を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 857
- Best CV score: 2.340703597440808
- Wall time: 9min 20s
- 評価結果  : 2.3554087

#### 2(156)  ...merge12のパラメータをチューニング
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 835
- Best CV score: 2.3419206656914
- Wall time: 9min 7s
- 評価結果  : 2.3555299

#### 2(156)  ...merge12(2017のコースデータを付加) 
- 'learning_rate' : 0.1,
- Best num_boost_round: 121
- Best CV score: 2.344104226715559
- Wall time: 12min 30s
- 評価結果  : 2.3564326

#### 1(143)  ...球種予測の11と同じデータ 
- 'learning_rate' : 0.1,
- Best num_boost_round: 122
- Best CV score: 2.3445089434874253
- Wall time: 12min 2s (local)
- 評価結果  : 2.3562648